In [ ]:
! pip install langchain transformers accelerate bitsandbytes
!pip install langchain_experimental SentencePiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 798.0/798.0 kB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.5/215.5 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.0/163.0 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.3/803.3 kB 59.6 MB/s eta 0:00:00
  Attempting uninstall: langchain
    Found existing installation: langchain 0.1.0
    Uninstalling langchain-0.1.0:
      Successfully uninstalled langchain-0.1.0


## LLM

## DB

Connect to a SQLite DB.

To create this particular DB, you can use the code and follow the steps shown [here](https://github.com/facebookresearch/llama-recipes/blob/main/demo_apps/StructuredLlama.ipynb).

In [ ]:
from langchain_community.utilities import SQLDatabase
from google.colab import drive
#drive.mount('/content/drive')
db = SQLDatabase.from_uri("sqlite:///roster.db", sample_rows_in_table_info=0)


def get_schema(_):
    return db.get_table_info()


def run_query(query):
    return db.run(query)



In [ ]:
print(db.get_table_info())
print(db.run("select count(*) from roster"))
print(type(db.run("select count(*) from roster")))


CREATE TABLE roster (
	"Overall_rank" INTEGER, 
	"Country" TEXT, 
	"Score" REAL, 
	"GDP_per_capita" REAL, 
	"Social_support" REAL, 
	"Healthy_life_expectancy" REAL, 
	"Freedom_to_make_life_choices" REAL, 
	"Generosity" REAL, 
	"Perceptions_of_corruption" REAL
)
[(156,)]
<class 'str'>


In [ ]:
print(db.run("SELECT Country FROM roster WHERE GDP_per_capita = (SELECT MAX(GDP_per_capita) FROM roster)"))

[('Qatar',)]


In [ ]:
#!pip install SentencePiece
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Initialize the tokenizer from Hugging Face Transformers library
tokenizer = T5Tokenizer.from_pretrained('t5-small')

# Load the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = T5ForConditionalGeneration.from_pretrained('cssupport/t5-small-awesome-text-to-sql')



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.32k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/2.37k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/242M [00:00<?, ?B/s]

In [ ]:
#https://huggingface.co/cssupport/t5-small-awesome-text-to-sql?text=what+is+the+count+of+records%3F

#model = model.to(device)
#model.eval()

def generate_sql(input_prompt):
    # Tokenize the input prompt
    inputs = tokenizer(input_prompt, padding=True, truncation=True, return_tensors="pt")
    #.to(device)

    # Forward pass
    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=512)

    # Decode the output IDs to a string (SQL query in this case)
    generated_sql = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(generated_sql)
    return generated_sql

In [ ]:

input_prompt = "tables:\n" + db.get_table_info() + "\n" + "query for:" + "What is the GDP per capita for finland?"

generated_sql = generate_sql(input_prompt)

print(f"The generated SQL query is: {generated_sql}")

In [ ]:
input_prompt = "tables:\n" + db.get_table_info() + "\n" + "query for:" + "which country has the highest GDP per capita?"

generated_sql = generate_sql(input_prompt)

print(f"The generated SQL query is: {generated_sql}")

SELECT MIN(Overall_rank) FROM roster WHERE GDP_per_capita = (SELECT MAX(GDP_per_capita) FROM roster)
The generated SQL query is: SELECT MIN(Overall_rank) FROM roster WHERE GDP_per_capita = (SELECT MAX(GDP_per_capita) FROM roster)


In [ ]:
input_prompt = "tables:\n" + db.get_table_info() + "\n" + "query for:" + "what is the name of country with the highest GDP per capita?"

generated_sql = generate_sql(input_prompt)

print(f"The Answer is: {run_query(generated_sql)}")

SELECT MAX(Overall_rank) FROM roster WHERE Country = "Country" GROUP BY Country ORDER BY MAX(GDP_per_capita) DESC LIMIT 1
The Answer is: [(29,)]


In [ ]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
from transformers import pipeline

pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_new_tokens=500)
hf = HuggingFacePipeline(pipeline=pipe)

## Query a SQL Database

Follow the runnables workflow [here](https://python.langchain.com/docs/expression_language/cookbook/sql_db).

In [ ]:
# Prompt
from langchain.prompts import ChatPromptTemplate
input_prompt = "tables:\n {schema} \n " + " query for: {question}"


prompt = ChatPromptTemplate.from_template(input_prompt)
llm=hf
# Chain to query
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

sql_response = (
    RunnablePassthrough.assign(schema=get_schema)
    | prompt
    | llm
    | StrOutputParser()
)
#llm.bind(stop=["\nSQLResult:"])
sql_response.invoke({"question": "which country has the highest GDP per capita?"})

'SELECT Country FROM roster WHERE GDP_per_capita = (SELECT MAX(GDP_per_capita) FROM roster)'